In [ ]:
import torch
import torchvision
import torch.nn.functional as F
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, Subset
from torch.optim.lr_scheduler import ReduceLROnPlateau, OneCycleLR
from efficientnet_pytorch import EfficientNet
from sklearn.metrics import accuracy_score, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
import cv2

import os
import time
import datetime
import warnings
from pathlib import Path

In [ ]:
warnings.simplefilter('ignore')
torch.manual_seed(47)
np.random.seed(47)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class MelanomaDataset(Dataset):
    def __init__(self, df: str, data: np.array, train: bool = True, transforms = None):
        """
        Class initialization
        Args:
            df (str): path to pandas.DataFrame 
            data (np.ndarray): resized images data in a shape of (HxWxC)
            train (bool): flag of whether a training dataset is being initialized or testing one
            transforms: image transformation method to be applied
            
        """
        self.df = pd.read_csv(df)
        self.data = data
        self.transforms = transforms
        self.train = train
        if self.train:
            self.y = self.df['target'].values
        
    def __getitem__(self, index):
        x = self.data[index]

        if self.transforms:
            x = transforms.ToPILImage()(x)
            x = self.transforms(x)
            
        if self.train:
            y = self.df.iloc[index]['target']
            return x, y
        else:
            return x
    
    def __len__(self):
        return len(self.data)
    
    
class Net(nn.Module):
    def __init__(self, arch):
        super(Net, self).__init__()
        self.arch = arch
        self.arch.fc = nn.Linear(in_features=2048, out_features=1, bias=True)
        #self.arch._fc = nn.Linear(in_features=1280, out_features=1, bias=True)
        
    def forward(self, x):
        """
        No sigmoid in forward because we are going to use BCEWithLogitsLoss
        Which applies sigmoid for us when calculating a loss
        """
        x = self.arch(x)
        return x

In [ ]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

train = MelanomaDataset(df='/home/kitamura/dataset/Melanoma/train.csv', 
                        data=np.load('./x_train_224.npy'),
                        transforms=transform)
skf = StratifiedKFold(n_splits=5, random_state=47, shuffle=True)
test = MelanomaDataset(df='/home/kitamura/dataset/Melanoma/test.csv', 
                       data=np.load('./x_test_224.npy'), 
                       train=False,
                       transforms=transform)

In [ ]:
epochs = 10  # Number of epochs to run
model_path = 'model.pth'  # Path and filename to save model to
es_patience = 3  # Early Stopping patience - for how many epochs with no improvements to wait

oof = np.zeros((len(train), 1))  # Out Of Fold predictions
preds = torch.zeros((len(test), 1), dtype=torch.float32, device=device)  # Predictions for test test

# We stratify by target value, thus, according to sklearn StratifiedKFold documentation
# We can fill `X` with zeroes of corresponding length to use it as a placeholder
# since we only need `y` to stratify the data
for fold, (train_idx, val_idx) in enumerate(skf.split(X=np.zeros(len(train)), y=train.y), 1):
    print('=' * 20, 'Fold', fold, '=' * 20)
    
    best_val = None  # Best validation score within this fold
    patience = es_patience  # Current patience counter
    #arch = EfficientNet.from_pretrained('efficientnet-b0')  # Going to use efficientnet-b0 NN architecture
    arch = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50', pretrained=True) 
    model = Net(arch=arch)  # New model for each fold
    model = model.to(device)
    optim = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.BCEWithLogitsLoss()
    
    train_loader = DataLoader(dataset=Subset(train, train_idx), batch_size=70, shuffle=True)
    val_loader = DataLoader(dataset=Subset(train, val_idx), batch_size=50, shuffle=False)
    test_loader = DataLoader(dataset=test, batch_size=50, shuffle=False)
    
    for epoch in range(epochs):
        start_time = time.time()
        correct = 0
        epoch_loss = 0
        model.train()
        
        for idx, (x, y) in enumerate(train_loader):
            x = torch.tensor(x, device=device, dtype=torch.float32)
            y = torch.tensor(y, device=device, dtype=torch.float32)
            optim.zero_grad()
            z = model(x)
            loss = criterion(z, y.unsqueeze(1))
            loss.backward()
            optim.step()
            pred = torch.round(torch.sigmoid(z))  # round off sigmoid to obtain predictions
            correct += (pred.cpu() == y.cpu().unsqueeze(1)).sum().item()  # tracking number of correctly predicted samples
            epoch_loss += loss.item()
            print("\r{} / {}".format(idx * 70, len(train_idx)), end="")
        print("finish training")
        
        train_acc = correct / len(train_idx)

        model.eval()  # switch model to the evaluation mode
        val_preds = torch.zeros((len(val_idx), 1), dtype=torch.float32, device=device)
        with torch.no_grad():  # Do not calculate gradient since we are only predicting
            # Predicting on validation set
            for j, (x_val, y_val) in enumerate(val_loader):
                x_val = torch.tensor(x_val, device=device, dtype=torch.float32)
                y_val = torch.tensor(y_val, device=device, dtype=torch.float32)
                z_val = model(x_val)
                val_pred = torch.sigmoid(z_val)
                val_preds[j*x_val.shape[0]:j*x_val.shape[0] + x_val.shape[0]] = val_pred
            val_acc = accuracy_score(train.df.iloc[val_idx]['target'].values, torch.round(val_preds.cpu()))
            val_roc = roc_auc_score(train.df.iloc[val_idx]['target'].values, val_preds.cpu())
            
            print('Epoch {:03}: | Loss: {:.3f} | Train acc: {:.3f} | Val acc: {:.3f} | Val roc_auc: {:.3f} | Training time: {}'.format(
            epoch + 1, 
            epoch_loss, 
            train_acc, 
            val_acc, 
            val_roc, 
            str(datetime.timedelta(seconds=time.time() - start_time))))
            
            # During the first iteration (first epoch) best validation is set to None
            if not best_val:
                best_val = val_roc  # So any validation roc_auc we have is the best one for now
                torch.save(model, model_path)  # Saving the model
                continue
                
            if val_roc >= best_val:
                best_val = val_roc
                patience = es_patience  # Resetting patience since we have new best validation accuracy
                torch.save(model, model_path)  # Saving current best model
            else:
                patience -= 1
                if patience == 0:
                    print('Early stopping. Best Val roc_auc: {:.3f}'.format(best_val))
                    break
                
    model = torch.load(model_path)  # Loading best model of this fold
    model.eval()  # switch model to the evaluation mode
    val_preds = torch.zeros((len(val_idx), 1), dtype=torch.float32, device=device)
    with torch.no_grad():
        # Predicting on validation set once again to obtain data for OOF
        for j, (x_val, y_val) in enumerate(val_loader):
            x_val = torch.tensor(x_val, device=device, dtype=torch.float32)
            y_val = torch.tensor(y_val, device=device, dtype=torch.float32)
            z_val = model(x_val)
            val_pred = torch.sigmoid(z_val)
            val_preds[j*x_val.shape[0]:j*x_val.shape[0] + x_val.shape[0]] = val_pred
        oof[val_idx] = val_preds.cpu().numpy()
        
        # Predicting on test set
        for i, x_test in enumerate(test_loader):
            x_test = torch.tensor(x_test, device=device, dtype=torch.float32)
            z_test = model(x_test)
            z_test = torch.sigmoid(z_test)
            preds[i*x_test.shape[0]:i*x_test.shape[0] + x_test.shape[0]] += z_test / skf.n_splits